In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from keras.models import *
from keras.layers import *
from keras.datasets import mnist
import os

In [ ]:
OUT_DIR = './CNN_out'
img_shape = (28, 28, 1)
epochs = 100000
batch_size = 128
noise = 100
sample_interval = 100       # 100마다 샘플뽑아서 저장

In [ ]:
(x_train, _), (_, _) = mnist.load_data()     # _ == ytrain, xtest, ytest 필요없어서 _로 표시
print(x_train.shape)

x_train = x_train / 127.5 - 1   # -1 에서 1 사이의 값이 나옴
x_train = np.expand_dims(x_train, axis=3)   # expand_dims : 익스펜드 디멘션 -> 차원을 하나 늘려라
print(x_train.shape)


(60000, 28, 28)
(60000, 28, 28, 1)


In [ ]:
generator = Sequential()
generator.add(Dense(256*7*7, input_dim=noise))  # 레이어 100개
generator.add(Reshape((7, 7, 256)))     # 256 장을 합치고 있음
generator.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))  # Conv2DTranspose 사이즈 커짐 - conv + maxpool 이라고 생각하면 됨
generator.add(BatchNormalization())
generator.add(LeakyReLU(alpha=0.01))
generator.add(Conv2DTranspose(64, kernel_size=3, strides=1, padding='same'))
generator.add(LeakyReLU(alpha=0.01))
generator.add(BatchNormalization())
generator.add(Conv2DTranspose(1, kernel_size=3, strides=2, padding='same'))  
generator.add(Activation('tanh'))
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12544)             1266944   
                                                                 
 reshape (Reshape)           (None, 7, 7, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 14, 14, 128)      295040    
 nspose)                                                         
                                                                 
 batch_normalization (BatchN  (None, 14, 14, 128)      512       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 14, 14, 128)       0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 14, 14, 64)       7

In [ ]:
discriminator = Sequential()
discriminator.add(Conv2D(32, kernel_size=3, strides=2, padding='same', input_shape=img_shape))
discriminator.add(LeakyReLU(alpha=0.01))
discriminator.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
discriminator.add(LeakyReLU(alpha=0.01))
discriminator.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
discriminator.add(LeakyReLU(alpha=0.01))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 32)        320       
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 14, 14, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 7, 7, 64)          0         
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 4, 4, 128)         0         
                                                                 
 flatten (Flatten)           (None, 2048)             

In [ ]:
discriminator.compile(loss='binary_crossentropy', optimizer='adam',
                      metrics=['accuracy'])
discriminator.trainable = False

gan_model = Sequential()
gan_model.add(generator)
gan_model.add(discriminator)
gan_model.summary()
gan_model.compile(loss='binary_crossentropy', optimizer='adam')

real = np.ones((batch_size, 1))     # np.ones : 1로 채워진 행렬로 만들어 주는 것
fake = np.zeros((batch_size, 1))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 28, 28, 1)         1637121   
                                                                 
 sequential_1 (Sequential)   (None, 1)                 94721     
                                                                 
Total params: 1,731,842
Trainable params: 1,636,737
Non-trainable params: 95,105
_________________________________________________________________


In [ ]:
for epoch in range(epochs):
    idx = np.random.randint(0, x_train.shape[0], batch_size)    # 0-59999 랜덤으로 뽑아냄
    real_imgs = x_train[idx]

    z = np.random.normal(0, 1, (batch_size, noise))
    fake_imgs = generator.predict(z)

    d_hist_real = discriminator.train_on_batch(real_imgs, real)    # train_on_batch: 1회만 하고 그만둠
    d_hist_fake = discriminator.train_on_batch(fake_imgs, fake)

    d_loss, d_acc = np.add(d_hist_fake, d_hist_real) * 0.5  # 평균을 구함

 
    if epoch % 2 ==0:
        z = np.random.normal(0, 1, (batch_size, noise))
        gan_hist = gan_model.train_on_batch(z, real) # 1이라고 답하게 학습

    if epoch % sample_interval ==0:
        print('%d, [D loss: %f, acc.: %.2f%%], [G loss: %f]'%(
            epoch, d_loss, d_loss, gan_hist))
        row = col = 4
        z = np.random.normal(0, 1 ,(row*col, noise))
        fake_imgs = generator.predict(z)
        fake_imgs = 0.5 * fake_imgs + 0.5

        _, axs = plt.subplots(row, col, figsize=(5, 5),sharey =True, sharex=True)
        cont = 0
        for i in range(row):
            for j in range(col):
                axs[i, j].imshow(fake_imgs[cont, :, :, 0], cmap='gray')
                axs[i, j].axis('off')
                cont += 1
        path = os.path.join(OUT_DIR, 'img-{}'.format(epoch + 1))
        plt.savefig(path)
        plt.close()








스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
4/4 [==============================] - 0s 4ms/step
12300, [D loss: 0.148593, acc.: 0.15%], [G loss: 3.848994]
4/4 [==============================] - 0s 3ms/step
12400, [D loss: 0.198835, acc.: 0.20%], [G loss: 3.438805]
4/4 [==============================] - 0s 3ms/step
12500, [D loss: 0.142393, acc.: 0.14%], [G loss: 3.261203]
4/4 [==============================] - 0s 6ms/step
12600, [D loss: 0.201393, acc.: 0.20%], [G loss: 3.754643]
4/4 [==============================] - 0s 4ms/step
12700, [D loss: 0.128397, acc.: 0.13%], [G loss: 4.552035]
4/4 [==============================] - 0s 6ms/step
12800, [D loss: 0.158147, acc.: 0.16%], [G loss: 4.174702]
4/4 [==============================] - 0s 3ms/step
12900, [D loss: 0.260435, acc.: 0.26%], [G loss: 3.939409]
4/4 [==============================] - 0s 5ms/step
13000, [D loss: 0.205009, acc.: 0.21%], [G loss: 2.878250]
4/4 [==============================] - 0s 4ms/step
13100, [D loss: 0.245383, acc.: 0

KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')